In [22]:
import os
import sentinelhub
from src.data import load_sentinelhub_credentials, load_eval_script
from dotenv import load_dotenv
import geopandas as gpd
from tqdm.notebook import tqdm
import rasterio
from PIL import Image
#

In [23]:
load_dotenv()
config = load_sentinelhub_credentials()
evalscript_ndvi = load_eval_script("evalscript.js")

In [ ]:
city_gdf = gpd.read_file("../datasets/villes_villages_fleuris.shp")
city_gdf = city_gdf.to_crs(epsg=4326)
city_gdf.head()

In [25]:
resolution = 10  # in meters
# iterate over the cities
for i, row in city_gdf.iterrows():
    city_name = row["commune"]
    aoi_coords_wgs84 = row["geometry"].bounds
    aoi_bbox = sentinelhub.BBox(
        bbox=aoi_coords_wgs84, crs=sentinelhub.CRS.WGS84)
    aoi_size = sentinelhub.bbox_to_dimensions(aoi_bbox, resolution=resolution)

    if aoi_size[0] > 2500 or aoi_size[1] > 2500:
        raise ValueError(
            "Image dimensions are too large. Please specify a smaller area of interest.")

In [26]:
# byoc collection id for quarterly mosaic
byoc_id = "5460de54-082e-473a-b6ea-d5cbe3c17cca"
S2l3_cloudless_mosaic = sentinelhub.DataCollection.define_byoc(
    collection_id=byoc_id
)

In [ ]:
resolution = 10  # in meters
# iterate over the cities
os.makedirs("../datasets/ndvi_geotiff", exist_ok=True)

for i, row in tqdm(city_gdf.iterrows(), total=city_gdf.shape[0]):
    aoi_coords_wgs84 = row["geometry"].bounds
    aoi_bbox = sentinelhub.BBox(
        bbox=aoi_coords_wgs84, crs=sentinelhub.CRS.WGS84)

    aoi_size = sentinelhub.bbox_to_dimensions(aoi_bbox, resolution=resolution)

    request_ndvi = sentinelhub.SentinelHubRequest(
        evalscript=evalscript_ndvi,
        input_data=[
            sentinelhub.SentinelHubRequest.input_data(
                data_collection=S2l3_cloudless_mosaic,
                time_interval=("2022-06-01", "2022-10-01"),
            )
        ],
        responses=[sentinelhub.SentinelHubRequest.output_response(
            # default to get the RGB approximation, index to get the NDVI
            "index", sentinelhub.MimeType.TIFF)],
        bbox=aoi_bbox,
        size=aoi_size,
        config=config,
        data_folder="./data",
    )
    response = request_ndvi.get_data(save_data=True)
    ndvi_img = response[
        0]
    # Construct the file path for the GeoTIFF

    ndvi_img_path = f"../datasets/ndvi_geotiff/{row["code_insee"]}.tiff"
    transform = rasterio.transform.from_bounds(
        *aoi_coords_wgs84, *aoi_size
    )
    crs = rasterio.crs.CRS.from_epsg(4326)

    # Save the NDVI image as a GeoTIFF with geospatial metadata
    with rasterio.open(
        ndvi_img_path,
        'w',
        driver='GTiff',
        height=ndvi_img.shape[0],
        width=ndvi_img.shape[1],
        count=1,  # Number of bands
        dtype=ndvi_img.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(ndvi_img, 1)  # Write the NDVI data to the first band